# Notebook used to obtain the training and test datasets

In [1]:
import ee
import geemap

import json
from ipyleaflet import GeoJSON

import os
import pickle 

In [8]:
Map = geemap.Map(center=[-15, -57] )

In [3]:
# Use this information to set polygon label and color property 
legend_dict = {
    '0 - Water': '3388ff',
    '1 - Bare soil': 'e72a08',
    '2 - Forest': '3ba702',
    '3 - Annual crop': 'e4f406',
    '4 - Perennial crop': '44ff00',
    '5 - Pasture': 'c88d0e',
    '6 - Urban area': 'bab7b0'
}

## Step 1: zoom in to a area of interest and draw a small polygon

In [9]:
Map

Map(center=[-15, -57], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

## Step 2: Run the next cell to define and display the image object as an Sentinel-2 image

In [11]:
image = (ee.ImageCollection('COPERNICUS/S2_SR')
         .filterBounds(Map.user_rois.geometry())
         .filterDate('2020-09-01', '2020-12-31')
         .sort('CLOUDY_PIXEL_PERCENTAGE')
         .first()
         )

# Add the image layer to the map and display it.
Map.add_ee_layer(
    image, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2000}, 'sentinel')


## Step 3: Set and draw polygon properties accordingly with the land cover classes

- Before start this process take care to clean all previous polygons with had drawn

##### For more details see the video below:
geemap.show_youtube("https://youtu.be/VWh5PxXPZw0")

# Make the training dataset.

In [12]:
if Map.user_rois is not None:
    aoi = Map.user_rois
    print(aoi.getInfo())

{'type': 'FeatureCollection', 'columns': {'system:index': 'String'}, 'features': [{'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[-54.574371, -15.427158787467349], [-54.598928918241555, -15.430530610855428], [-54.62153426815929, -15.440377997472881], [-54.64038920968418, -15.455917938975814], [-54.65399325780134, -15.475914568064514], [-54.66126260423133, -15.498777187957492], [-54.66161667381733, -15.522686569195363], [-54.6550249392486, -15.545739561039282], [-54.64201008214709, -15.566100569311669], [-54.623607020860526, -15.582147848255545], [-54.60128086687716, -15.592602906669784], [-54.57681023312644, -15.596632629140894], [-54.55214520275903, -15.593915872316812], [-54.529251406197396, -15.584669146544432], [-54.50995283748738, -15.569629301021786], [-54.49578614873132, -15.549994616280339], [-54.487878182635406, -15.527329073491137], [-54.486856537195365, -15.503437530265206], [-54.49280019750091, -15.48022184263001], [-54.50523397855468, -15.459529451168

In [13]:
lat = str(aoi.first().getInfo()['geometry']['coordinates'][0][0][0]).replace('-', "").replace('.','')
lon = str(aoi.first().getInfo()['geometry']['coordinates'][0][0][1]).replace('-', "").replace('.','')

In [ ]:
# Use these bands for prediction.
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']

# This property of the table stores the land cover labels.
label = 'landcover'

# Overlay the points on the imagery to get training.
data_points = image.select(bands).sampleRegions(**{
  'collection': aoi,
  'properties': [label],
  'scale': 10,
  'tileScale':16
})

In [35]:
print(data_points.size().getInfo())

27454


#### Save training points

- if you retrieving training data run the cell below

In [ ]:
#Save aoi for a given image
with open('..data/train/polygons/aoi_'+lat+'_'+lon+'.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(aoi.filter(ee.Filter.notNull(['landcover']) ), f)

In [ ]:
with open('../data/train/data_'+lat+'_'+lon+'.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(data_points.filter(ee.Filter.notNull(['landcover']) ), f)

#### Save external validation points
- to save the external validation data set run the cell below

In [ ]:
#Save aoi for a given image
with open('..data/test/polygons/aoi_'+lat+'_'+lon+'.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(aoi.filter(ee.Filter.notNull(['landcover']) ), f)

In [ ]:
with open('../data/test/data_'+lat+'_'+lon+'.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(data_points.filter(ee.Filter.notNull(['landcover']) ), f)

## Step 4: repeat this process for other region and image